# Viết hàm trong Python

## Giới thiệu về hàm

Tương tự như R, Python cho phép sử dụng hàm linh hoạt với các tham số và trả ra kết quả định sẵn. Có 1 số lưu ý như sau:

- Hàm bắt đầu với `def`
- Các tham số mặc định được sử dụng tương tự như R. VD `def plus(a, b = 2)`
- Trong hàm cho phép ghi chú document của hàm trong ba dấu ngoặc kép, được gọi là `docs string`
- Các tham số chưa biết định dạng tuple (tương ứng với `...` trong R) được sử dụng với argument `*args` (argument)
- Các tham số với keyword được sử dụng với argument `**kargs` (keyword argument)

In [1]:
def plus_1(a, b = 2):
    """Trả ra kết quả hàm tổng"""
    return a + b

In [2]:
plus_1(4)

6

In [3]:
?plus_1

Signature: plus_1(a, b=2)
Docstring: Trả ra kết quả hàm tổng
File:      c:\users\anhhd3\appdata\local\temp\ipykernel_16704\2568156839.py
Type:      function

In [4]:
def plus_2(*args):
    return np.sum(args)

In [5]:
plus_2(3,4,5)

NameError: name 'np' is not defined

**Ví dụ hàm trả ra n số Fibonacci đầu tiên**

In [6]:
def fib_n(n):
    if n == 1:
        my_list = list([0])
    elif n == 2:
        my_list = list([0, 1])
    else: 
        my_list = list([0, 1])
        f0 = 0
        f1 = 1
        my_range = range(n-2)
        for i in my_range:
            f2 = f1 + f0
            f0 = f1
            f1 = f2
            my_list.append(f2)
    return(my_list)

In [7]:
fib_n(9)

[0, 1, 1, 2, 3, 5, 8, 13, 21]

---

Ví dụ giữa `arg*` và `karg**`

In [8]:
def printall(*args):
  print(args)

In [9]:
printall(1, 2, '3')

(1, 2, '3')


In [10]:
#| eval: false
printall(1, 2, x = '3')

TypeError: printall() got an unexpected keyword argument 'x'

In [11]:
def print2(*arg, **karg):
  print(arg, karg)

## Hàm summary theo group

In [12]:
def load_iris_df():
    """
    Load iris data in the form of dataframe with grouping variable.
    The data set is used to ilustrate different functions of VPBank package
    """
    
    import numpy as np
    import pandas as pd
    from sklearn.datasets import load_iris
    iris = load_iris()
    ref_species = pd.DataFrame({
        'species_code' : [0, 1, 2],
        'species' : iris.target_names
    })
    # Sửa lại name
    name = pd.Series(iris.feature_names)
    name = name.str.replace('\\(cm\\)', '')
    name = name.str.strip().str.replace(' ', '_')
    iris_df = pd.DataFrame(iris.data,
                           columns = name)
    iris_df['species_code'] = iris.target
    iris_df = pd.merge(iris_df, ref_species, how = 'left').drop('species_code', 1)
    iris_df['group'] = np.random.randint(1,3, 150)
    iris_df.group = iris_df.group.astype(object)
    return iris_df

In [20]:
import pandas as pd
import numpy as np
iris_df = pd.read_csv("99_dataset/iris.csv")
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [26]:
iris_df['group'] = np.repeat([1, 2], 75)

In [30]:
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,group
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


In [31]:
iris_df.groupby(['species'])['sepal_length'].agg({'mean' : np.mean})

SpecificationError: nested renamer is not supported

In [29]:
iris_df.groupby(['species', 'group'])['sepal_length'].agg({'mean' : np.mean,
    'median': np.median})

SpecificationError: nested renamer is not supported

In [ ]:
np.quantile(iris_df['sepal_length'], [0.75, 0.9])

In [ ]:
# Tạo hàm tính percentile
def percentile(n):
    def percentile_(x):
        return np.percentile(x,n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [ ]:
iris_df.groupby('species')['sepal_length'].agg({
    'mean',
    np.median,
    percentile(90),
    percentile(25),
    'count'
})

In [ ]:
def analytics_summarise(data, value_var, group_var):
    """Quickly summarise statistics of one measures by different groups
    
    Args: 
    
    - data: dataframe to analyze
    - value_var: variable to analyze
    - group_var: list of grouping variables
    
    Returns:
    
    - A dataframe of summarise statistics
    
    """
    
    def _percentile(n):
        def percentile_(x):
            return np.percentile(x,n)
        percentile_.__name__ = 'percentile_%s' % n
        return percentile_

    result = data.groupby(group_var)[value_var].agg({
        'n' : 'count',
        'mean' : np.mean,
        'min' : np.min,
        'q25' : _percentile(25),
        'median' : np.median,
        'q75' : _percentile(75),
        'q90' : _percentile(90),
        'max' : np.max})
    return result

In [ ]:
analytics_summarise(iris_df, 'sepal_length', ['species', 'group'])

In [ ]:
# boxplot
def _draw_boxplot(data, value_var, group_var):
    import seaborn as sns
    from matplotlib import pyplot
    # Set retina resolution for chart
    %config InlineBackend.figure_format = 'retina'
    sns.set_style("ticks")
    my_plot = sns.boxplot(x = "species", y = "sepal_length", data = iris_df)
    sns.despine()     
    return my_plot

In [ ]:
plot = _draw_boxplot(iris_df, 'sepal_length', 'species')

In [ ]:
import matplotlib.pyplot as plt
plot.set_yscale('log')

In [ ]:
plot.get_figure()

In [ ]:
plot.set(title = 'My plot')
plot.get_figure()

## ANOVA Tukey HSD

In [ ]:
def _anova_turkey(data, value_var, group_var):   
    from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                             MultiComparison)

    MultiComp = MultiComparison(data[value_var],
                                data[group_var])

    # Show all pair-wise comparisons:

    # Print the comparisons
    result = MultiComp.tukeyhsd().summary()
    print('Comparison of ' + value_var + ' by ' + group_var)
    print('===========================================')
  #  print(result)
    return result

In [ ]:
_anova_turkey(iris_df, 'sepal_length', 'species')

## Compare group

In [ ]:
del [analytics_summarise, anova_turkey, compare_group, draw_boxplot]

In [ ]:
def compare_group(data, value_var, group_var):
    
    # Declare class for result
    
    class compare_result(object):
        def __init__(self, summary_table, aov_model, box_plot):
            self.summary_table = summary_table
            self.aov_model = aov_model
            self.box_plot = box_plot

    # Function to analyse data
    
    def analytics_summarise(data, value_var, group_var):
        """Summarise statistics"""
   
        def _percentile(n):
            def percentile_(x):
                return np.percentile(x,n)
            percentile_.__name__ = 'percentile_%s' % n
            return percentile_

        result = data.groupby(group_var)[value_var].agg({
            'n' : 'count',
            'mean' : np.mean,
            'min' : np.min,
            'q25' : _percentile(25),
            'median' : np.median,
            'q75' : _percentile(75),
            'q90' : _percentile(90),
            'max' : np.max})
        return result
    
    # Draw box plot
    def draw_boxplot(data, value_var, group_var):
        import seaborn as sns
        from matplotlib import pyplot
        # Set retina resolution for chart
        %config InlineBackend.figure_format = 'retina'
        sns.set_style("ticks")
        my_plot = sns.boxplot(x = data[group_var], y = data[value_var], data = data)
        sns.despine()     
        return my_plot
    
    # AOV model
    def aov_turkey(data, value_var, group_var):
        from statsmodels.stats.multicomp import (pairwise_tukeyhsd, MultiComparison)
        MultiComp = MultiComparison(data[value_var],
                                        data[group_var])
        result = MultiComp.tukeyhsd().summary()
        return result     
    
    return compare_result(
        summary_table = analytics_summarise(data, value_var, group_var),
        aov_model = aov_turkey(data, value_var, group_var),
        box_plot = draw_boxplot(data, value_var, group_var))

In [ ]:
stat_result = compare_goup(data = iris_df, group_var = 'species', value_var = 'sepal_length')

In [ ]:
stat_result.aov_model